In [ ]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import os

In [ ]:
# matplotlib configuration
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["axes.labelsize"] = 14
plt.rcParams["axes.labelweight"] = "bold"

In [ ]:
def readTimeFiles(inFileName, inMode, numTests, delta, tCount=""):
    baseDir = "time-tests"
    # Format of the timing files: $inFileName-$inMode$tCount-d$delta-run$i.out
    times = []
    for i in range(numTests):
        with open("{}/{}-{}{}-d{}-run{}.out".format(baseDir, inFileName, inMode, tCount, delta, i)) as f:
            times.append(float(f.read()))
    return times
def isParallel(mode):
    return mode.__contains__("parallel")
def readModesMean(benchDir, fileName, modes, numTimes, deltas, ts):
    modeTime = dict()
    for mode in modes:
        for delta in deltas:
            if isParallel(mode):
                for tCount in ts:
                    times = readTimeFiles(fileName, mode, numTimes, delta, tCount)
                    modeTime[mode + "-d{}t{}".format(delta, tCount)] = np.mean(times)
            else:
                times = readTimeFiles(fileName, mode, numTimes)
                modeTime[mode] = np.mean(times)
    return modeTime
def runReadModesMean(benchDir, fileName, modes, numTimes, deltas, ts):
    modeTime = dict()
    for mode in modes:
        for delta in deltas:
            if isParallel(mode):
                for tCount in ts:
                    os.system("bash testTimeBenchmarks.sh {} {}/{} {} {} {} 1> /dev/null"\
                              .format(numTimes, benchDir, fileName, mode, delta, tCount))
                    times = readTimeFiles(fileName, mode, numTimes, delta, tCount)
                    modeTime[mode + "-d{}t{}".format(delta, tCount)] = np.mean(times)
            else:
                os.system("bash testTimeBenchmarks.sh {} {}/{} {} {} 1> /dev/null".format(numTimes, benchDir, fileName, mode, delta))
                times = readTimeFiles(fileName, mode, numTimes, delta)
                modeTime[mode] = np.mean(times)
    return modeTime

In [ ]:
modes = ["dijkstra", "parallel", "parallel-bucket-fusion"]
deltaForTs = 1000
tsForTs = np.arange(1, 9)
numTimesForTs = 10
modeTimePolblogsVarTs = runReadModesMean("benchmarks", "slashdot_posedges.mtx", modes,
                                                   numTimes=numTimesForTs, deltas=[deltaForTs], ts=tsForTs)
modeTimeFacebookVarTs = runReadModesMean("benchmarks", "facebook-wchars.txt", modes,
                                                   numTimes=numTimesForTs, deltas=[deltaForTs], ts=tsForTs)
modeTime08blocksVarTs = runReadModesMean("benchmarks", "08blocks.mtx", modes,
                                                   numTimes=numTimesForTs, deltas=[deltaForTs], ts=tsForTs)

In [ ]:
deltaRange = []
for i in range(4, 21):
    deltaRange.append(2 ** i)
threadsForDs = 4
numTimesForDs = 10
modeTimePolblogsVarDs = runReadModesMean("benchmarks", "slashdot_posedges.mtx", modes,
                                                   numTimes=numTimesForDs, deltas=deltaRange, ts=[threadsForDs])
modeTimeFacebookVarDs = runReadModesMean("benchmarks", "facebook-wchars.txt", modes,
                                                   numTimes=numTimesForDs, deltas=deltaRange, ts=[threadsForDs])
modeTime08blocksVarDs = runReadModesMean("benchmarks", "08blocks.mtx", modes,
                                                   numTimes=numTimesForDs, deltas=deltaRange, ts=[threadsForDs])

### Now plot these

In [ ]:
# Plot experiments varying by number of threads
def plotVarByThread(m, mode, delta, ts):
    if not isParallel(mode):
        plt.scatter([0], [m[mode]], label=mode)
    else:
        yAxis = []
        for t in ts:
            yAxis.append(m[mode + "-d{}t{}".format(delta, t)])
        plt.plot(ts, yAxis, label=mode)
def plotAllVarByThread(maps, plotTitles, modes):
    figureDir = "figures"
    for i in range(len(maps)):
        plt.figure()
        plt.title(plotTitles[i])
        plt.xlabel("Number of threads (delta={})".format(deltaForTs))
        plt.ylabel("Execution time (secs)")
        plt.ylim(min(maps[i].values()) * 0.9, max(maps[i].values()) * 1.1)
        for md in modes:
            plotVarByThread(maps[i], md, deltaForTs, tsForTs)
        plt.legend()
        plt.savefig(figureDir + "/" + plotTitles[i] + "VarTs")
        plt.show()

varTsMaps = [modeTimePolblogsVarTs, modeTimeFacebookVarTs, modeTime08blocksVarTs]
varTsPlotTitles = ["PolBlogs", "Facebook", "08blocks"]
plotAllVarByThread(varTsMaps, varTsPlotTitles, modes)

In [ ]:
# Plot experiments varying by delta
def plotVarByDelta(m, mode, deltas, t):
    if not isParallel(mode):
        plt.scatter([0], [m[mode]], label=mode)
    else:
        yAxis = []
        for d in deltas:
            yAxis.append(m[mode + "-d{}t{}".format(d, t)])
        plt.plot(deltas, yAxis, label=mode)
    plt.xscale("log")
    plt.yscale("log")
def plotAllVarByDelta(maps, plotTitles, modes):
    figureDir = "figures"
    for i in range(len(maps)):
        plt.figure()
        plt.title(plotTitles[i])
        plt.xlabel("Delta (#threads={})".format(threadsForDs))
        plt.ylabel("Execution time (secs)")
        plt.xlim(min(deltaRange) * 0.9, max(deltaRange) * 1.1)
        for md in modes:
            plotVarByDelta(maps[i], md, deltaRange, threadsForDs)
        plt.legend()
        plt.savefig(figureDir + "/" + plotTitles[i] + "VarDs")
        plt.show()

varDsMaps = [modeTimePolblogsVarDs, modeTimeFacebookVarDs, modeTime08blocksVarDs]
varDsPlotTitles = ["PolBlogs", "Facebook", "08blocks"]
plotAllVarByDelta(varDsMaps, varDsPlotTitles, modes)

## Now varying the way the main loops is scheduled

In [ ]:
# TODO: take the mean of a few runs, instead of just doing one
scheduleList = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]
times = []
for schedule in scheduleList:
    with open("time-tests/schedule{}-slash.out".format(schedule)) as f:
        times.append(float(f.read()))

In [ ]:
# Plot it
plt.figure()
plt.plot(scheduleList, times)
plt.xscale("log")
plt.yscale("log")
plt.savefig("figures/scheduleGraph")
plt.show()